In [45]:
import pymysql
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)

mysql_host = "142.132.134.21"
mysql_prot = 9030
mysql_user = "dw"
mysql_pd = "a64imrJwqKZc9jiU!*aCYi.xnh9*TVhhBmZi"
mysql_db = "dw"
connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db=mysql_db, port=mysql_prot, charset='utf8')
cursor = connection.cursor()
logging.info("connection created")


def getSqlResult(sql):
    cursor.execute(sql)
    res_rows = cursor.fetchall()
    # df = pd.DataFrame(res_rows)
    field_names = [i[0] for i in cursor.description]
    return res_rows,field_names

INFO:root:connection created


In [47]:
from datetime import datetime

def nft_balance_cost_query(start_blo,end_blo):
    sql = """
INSERT INTO dm.dm_nft_balance_cost_eth_di
SELECT token_address
    ,token_id
    ,account_address
    ,`value` as balance_value
    ,block_number
    ,transaction_hash
    ,txn_block
    ,cost_eth_value
    ,cost_ts
    ,trade_type
    ,is_scalp
    ,now() as etl_time
FROM
(
    SELECT a.token_address
        ,a.token_id
        ,a.account_address
        ,a.`value`
        ,a.block_number
        ,b.cost_eth_value
        ,b.trade_type
        ,b.transaction_hash
        ,b.block_number as txn_block
        ,b.ts as cost_ts
        ,b.is_scalp
        ,row_number() over(partition by a.token_address,a.token_id ORDER BY b.ts DESC) as rnk
    FROM 
    (
        SELECT a.token_address,a.token_id,a.account_address,a.`value`,a.block_number,FROM_UNIXTIME(b.`timestamp`) as block_ts
        FROM dw.dws_nft_balance_eth as a
        left join prod.blocks as b
        ON a.block_number = b.block_number
        AND b.`chain` = 'eth'
        INNER JOIN dw.dwm_nft_detail_ha as c
        ON a.token_address = c.token_address
        AND c.token_type = 'ERC721'
        WHERE a.`value` > 0
        AND a.`block_number` >= {start_blo}
        AND a.`block_number` < {end_blo}
    ) as a
    LEFT JOIN 
    (
    select to_address as account_address
        ,token_address
        ,token_ids as token_id
        ,token_num
        ,buyer_pay_eth_amount as cost_eth_value
        ,trade_time as ts
        ,transaction_hash
        ,block_number
        ,trade_type
        ,is_scalp
    from dw.dwb_nft_trade_eth_detail_hi
    where erc_standard not in ('ERC1155')
    union all
    select account_address
        ,token_address
        ,token_id
        ,mint_amount as token_num
        ,(gas_cost+mint_cost) as cost_eth_value
        ,mint_time as ts
        ,transaction_hash
        ,block_number
        ,'mint' as trade_type
        ,0 as is_scalp
    from dw.dwb_nft_mint_detail_eth_hi
    ) as b
    ON a.token_address = b.token_address
    AND a.token_id = b.token_id
) as t
WHERE t.rnk = 1
""".format(
    start_blo=start_blo,
    end_blo=end_blo
)
    return sql

In [34]:
# 等分数据回溯区间
bucket_sql = """
SELECT block_number
    ,records
    ,cum_records
    ,rnk
FROM
(
    SELECT block_number
        ,records
        ,cum_records
        ,rank() over(PARTITION BY floor(cum_records/1000000) ORDER BY cum_records%1000000 asc) as rnk
    FROM
    (
        SELECT block_number
            ,records
            ,sum(records) over(ORDER BY block_number asc) as cum_records
        FROM
        (
            select block_number
                ,count(*) as records
            from dw.dws_nft_balance_eth
            where `value` > 0
            GROUP BY 1
        ) as a
    ) as aa
) as t
WHERE rnk = 1
"""
sql_res,sql_fields = getSqlResult(bucket_sql)

In [48]:
block_bucket = []
for row in sql_res:
    block_bucket.append(row[0])
    
print(len(block_bucket),sql_res[-1])

1113 (16210619, 76, 111200073, 1)


In [49]:
from datetime import datetime
from datetime import timedelta

def backfill_data(block_bucket,start_blo=0):
    start_blo = start_blo if start_blo > 0 else block_bucket[0]
    for i in range(1,len(block_bucket)):
        end_blo = block_bucket[i]
        if end_blo < start_blo:
            continue
        else:
            print(start_blo, end_blo)
            sql = nft_balance_cost_query(start_blo,end_blo)
            cursor.execute(sql)
            start_blo = end_blo

backfill_data(block_bucket,start_blo=14975000)

14975000 14976582
14976582 14977711
14977711 14978803
14978803 14980355
14980355 14981959
14981959 14984321
14984321 14986362
14986362 14989075
14989075 14991133
14991133 14993750
14993750 14996202
14996202 14998187
14998187 15000473
15000473 15002970
15002970 15005439
15005439 15007677
15007677 15010574
15010574 15013505
15013505 15015940
15015940 15018781
15018781 15021118
15021118 15023812
15023812 15025652
15025652 15027325
15027325 15029502
15029502 15031086
15031086 15033086
15033086 15035730
15035730 15037629
15037629 15039574
15039574 15041191
15041191 15042786
15042786 15044942
15044942 15047608
15047608 15049895
15049895 15051899
15051899 15053983
15053983 15055506
15055506 15057585
15057585 15059857
15059857 15061690
15061690 15063477
15063477 15065160
15065160 15066921
15066921 15068687
15068687 15070553
15070553 15072282
15072282 15073555
15073555 15074666
15074666 15076602
15076602 15078254
15078254 15079972
15079972 15081616
15081616 15083796
15083796 15085644
15085644 1

1
2
3
